<a href="https://colab.research.google.com/github/Vaycold/tensorflow_tutorial/blob/main/%2318.%EB%B6%84%EC%82%B0%20%EC%A0%84%EB%9E%B5%EC%9D%84%20%EC%82%AC%EC%9A%A9%ED%95%9C%20%EB%AA%A8%EB%8D%B8%20%EC%A0%80%EC%9E%A5%20%EB%B0%8F%20%EB%B6%88%EB%9F%AC%EC%98%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 훈련루프와 함께 tf.distribute.Strategy 사용
    유연성을 높이고 훈련을 더 잘 제어할 수 있도록 사용자 정의 훈련 루프 사용.
    -> 모델과 훈련 부프를 디버깅 하기 쉬워짐

In [6]:
import tensorflow as tf
import numpy as np
import os 

# 패션 MNIST 데이터셋 다운로드

fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images[...,None]
test_images = test_images[...,None]
train_images.shape

(60000, 28, 28, 1)

In [8]:
# 정규화
train_images = train_images / np.float32(255)
test_images = test_images / np.float32(255)

train_images.max(), test_images.max()

(1.0, 1.0)

## 변수와 그래프를 분산하는 전략 만들기
    tf.distribute.MirroredStrategy
     : 모든 변수와 모델 그래프는 장치에 복제됨
     : 입력은 장치에 고르게 분배되어 들어감
     : 각 장치는 주어지는 입력에 대해서 손실과 그래디언트를 계산
     : 그래디언트들을 전부 더함으로써 모든 장치들 간 그래디언트들 동기화
     : 동기화 후 동일한 업데이트가 각 장치에 있는 변수의 복사본에 동일 적용

In [9]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [10]:
print( "# of devices : {}".format(strategy.num_replicas_in_sync))

# of devices : 1


## 입력 파이프 라인 설정

In [13]:
buffer_size = len(train_images)
batch_size_per_replica = 64
global_batch_size = batch_size_per_replica * strategy.num_replicas_in_sync
epochs = 10

In [14]:
# 데이터셋 만들고 배포
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(buffer_size).batch(global_batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(global_batch_size)

train_dist_dataset = strategy.experimental_distribute_dataset(train_dataset)
test_dist_dataset = strategy.experimental_distribute_dataset(test_dataset)

## 모델 만들기

In [17]:
def create_model() :
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation = 'relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(64, 3, activation = 'relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation = 'relu'),
        tf.keras.layers.Dense(10, activation = 'softmax')

    ])
    return model

In [18]:
# Create a checkpoint dictionary to store the checkpoints
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir,'ckpt')

In [ ]:
## 손실함수 정의
'''
 -  예제당 손실을 합산하고 합계를 global_batch_size로 나눔
 -  scale_loss = tf.reduce_sum(loss) * (1./global_batch_size) 또는 
 -  tf.nn.compute_average_loss 사용
 '''